In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import requests
import os
import re
import pandas as pd

In [2]:

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument('blink-settings=imagesEnabled=false') #不加載圖片提高效率
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
# my_options.add_argument("--lang=zh-TW")  #設定為正體中文


# 初始化 WebDriver
service = Service(executable_path="./chromedriver.exe")  # 替換為 ChromeDriver 的路徑
# 使用 Chrome 的 WebDriver
# my_service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(
    options = my_options,
    service = service
)
# driver.implicitly_wait(15)


In [3]:
i = 1
recipe_urls_total = []
while True:
    
    # 目標URL
    url = f'https://www.food.com/recipe?pn={i}'
    i += 1
    
    try:
        driver.get(url)
        # 等待網頁加載完成
        time.sleep(3)  # 根據需要調整時間
    except:
        print('URL not found!')
        break

    # 爬取食譜列表頁面的食譜URL
    recipes = driver.find_elements(By.CSS_SELECTOR, "h2.title")  # 替換為正確的CSS選擇器


    recipe_urls = [recipe.find_element(By.TAG_NAME, 'a').get_attribute('href') for recipe in recipes]
    
    recipe_urls_total += recipe_urls
    if i % 100 == 0:
        recipe_urls_df = pd.DataFrame({'RecipeURL': recipe_urls_total})
        recipe_urls_df.to_csv(f'./foodcom_url/recipe_url_{i}_pages.csv', index=False)
        recipe_urls_total = []

KeyboardInterrupt: 

# 以下程式碼未使用

In [4]:
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 10
    waitTime = 3
    
    # 嘗試找到按鈕
    load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.gk-aa-load-more')

            # 檢查按鈕是否可見並可點擊
    if load_more_button.is_displayed() and load_more_button.is_enabled():
        # 點擊按鈕
        load_more_button.click()
        print('Clicked "Load More" button')
        
        # 等待數秒以便新內容加載
        time.sleep(3)
    else:
        print('Load More button not clickable or not found.')
        # return

    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        )

        '''
        或是每次只滾動一點距離，
        以免有些網站會在移動長距離後，
        將先前移動當中的元素隱藏

        例如將上方的 script 改成:
        offset += 600
        '''

        

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        
        '''
        [補充]
        如果要滾動的是 div 裡面的捲軸，可以使用以下的方法
        document.querySelector('div').scrollTo({...})
        '''
        
        # (重要)強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        time.sleep(waitTime)
        # waitTime += 0.03
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return document.documentElement.scrollHeight;'
        );
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1
            
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        # if offset >= 6000:
        #     break
scroll()



Clicked "Load More" button


In [4]:
save_dir = r"C:\Users\USER\OneDrive\final_group_project\images"
# save_dir = r"C:\Users\student\OneDrive\final_group_project\images"
save_path = []
# 定義正則表達式模式
patterns = {
    "Calories": r"Calories:\s+([\d.]+)",
    "Calories from Fat": r"Calories from Fat\s+([\d.]+)\s+g",
    "Total Fat": r"Total Fat\s+([\d.]+)\s+g",
    "Saturated Fat": r"Saturated Fat\s+([\d.]+)\s+g",
    "Cholesterol": r"Cholesterol\s+([\d.]+)\s+mg",
    "Sodium": r"Sodium\s+([\d.]+)\s+mg",
    "Total Carbohydrate": r"Total Carbohydrate\s+([\d.]+)\s+g",
    "Dietary Fiber": r"Dietary Fiber\s+([\d.]+)\s+g",
    "Sugars": r"Sugars\s+([\d.]+)\s+g",
    "Protein": r"Protein\s+([\d.]+)\s+g",
}
# 定義包含非法字符的正則表達式模式
illegal_chars_pattern = r'[\/\\\?<>:\*\|"]'

column_names = ['Filename', 'Calories', 'Calories from Fat', 'Total Fat', 'Saturated Fat', 'Cholesterol', \
                'Sodium', 'Total Carbohydrate', 'Dietary Fiber', 'Sugars', 'Protein']
df = pd.DataFrame(columns=column_names)



# for recipe in recipes:
#     print(recipe.text)
#     print("============")

for i, recipe_url in enumerate(recipe_urls):
    print(f"i = {i}, recipe_url = {recipe_url}")
    try:
        driver.get(recipe_url)
        time.sleep(3)  # 等待頁面加載
    except:
        print('recipe_url not found')
    
    filename = driver.find_element(By.CSS_SELECTOR, "div > h1").text
    # 使用 re.sub() 函數去除非法字符
    filename = re.sub(illegal_chars_pattern, '', filename)
    filename = filename.replace(" ", "_")
    print(f"filename = {filename}")

    try:
        image = driver.find_element(By.CSS_SELECTOR, 'img[style*="--aspect-ratio: 5/4;"]')  # 替換為正確的CSS選擇器
        image_url = image.get_attribute('src')
        print('Image URL:', image_url)
    except:
        print('Image not found')
    # 爬取食譜圖片

# 下载图片
    response = requests.get(image_url)
    time.sleep(2)  # 等待頁面加載
    if response.status_code == 200:
        # 拼接保存路径
        save_path = os.path.join(save_dir, filename + '.jpg')
        # 保存图片到指定路径
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print('img save to:', save_path)
    else:
        print('download img failed!')

    nutrition_section_btn = driver.find_element(By.CSS_SELECTOR, 'button.facts__nutrition')  # 替換為正確的CSS選擇器
    nutrition_section_btn.click()

    nutrition = ''
    nutrition_facts = driver.find_elements(By.CSS_SELECTOR, 'p.recipe-nutrition__item.svelte-epeb0m')
    for i, nutrition_fact in enumerate(nutrition_facts):
        nutrition += nutrition_fact.text
        # print(nutrition_fact.text)
        # print("====================")
        # nutrition.append()


    # 提取數值
    nutrition_info = [filename + '.jpg']
    for key, pattern in patterns.items():
        match = re.search(pattern, nutrition)
        if match:
            nutrition_info.append(float(match.group(1)))

    # # 顯示結果
    # for nutrient, value in nutrition_info.items():
    #     print(f"{nutrient}: {value}")
    df.loc[len(df.index)] = nutrition_info


i = 0, recipe_url = https://www.food.com/recipe/barbs-gumbo-82288
filename = BARB'S_GUMBO
Image URL: https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_744,ar_5:4/v1/img/recipes/82/28/8/cB14froTlmpDfRF49wmQ_gumbo%2520SITE-3.jpg
img save to: C:\Users\USER\OneDrive\final_group_project\images\BARB'S_GUMBO.jpg
i = 1, recipe_url = https://www.food.com/recipe/bourbon-chicken-45809
filename = BOURBON_CHICKEN
Image URL: https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_744,ar_5:4/v1/img/recipes/45/80/9/MwuCd6HpQ5mDvn4OLRkA_0S9A9886.jpg
img save to: C:\Users\USER\OneDrive\final_group_project\images\BOURBON_CHICKEN.jpg
i = 2, recipe_url = https://www.food.com/recipe/best-banana-bread-2886
filename = BEST_BANANA_BREAD
Image URL: https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_744,ar_5:4/v1/img/recipes/28/86/wBZxNua1T8yfDwbfo9Fz_0S9A9315.jpg
img save to: C:\Users\USER\OneDrive\final_group_project\images\BEST_BANANA_BREAD.jpg
i = 3, recipe_url = https://www.f

In [3]:
# 關閉 WebDriver
driver.quit()